In [ ]:
# [Student: Your Name 1, Student: Your Name 2, etc.] Import necessary libraries
import pandas as pd
import numpy as np
import random

# --- Part B: Technical Tasks ---

# --- 1. Simulate Transaction Data (Revised) ---

# [Student: Your Name] Define the path to the Groceries dataset CSV file
# Ensure 'Groceries_dataset.csv' is in the same directory as your script/notebook.
data_path = 'Groceries_dataset.csv'

# [Student: Your Name] Load the dataset into a pandas DataFrame
# The dataset has columns: 'Member_number', 'Date', 'itemDescription'.
try:
    df_raw = pd.read_csv(data_path)
    # [Student: Your Name] Display the first few rows and info of the raw data to confirm its structure
    print("--- Raw Groceries Data Head ---")
    print(df_raw.head())
    print("\n--- Raw Groceries Data Info ---")
    df_raw.info()
except FileNotFoundError:
    print(f"Error: The file '{data_path}' was not found.")
    print("Please download 'Groceries_dataset.csv' from https://www.kaggle.com/datasets/balajikartheek/groceries-dataset")
    print("and place it in the correct directory.")
    # [Student: Your Name] Exit the script if the file is not found
    exit()

Error: The file 'Groceries_dataset.csv' was not found.
Please download 'Groceries_dataset.csv' from https://www.kaggle.com/datasets/balajikartheek/groceries-dataset
and place it in the correct directory.


: 